In [1]:
from rsna_retro.imports import *
from rsna_retro.metadata import *
from rsna_retro.preprocess import *
from rsna_retro.train import *

In [2]:
torch.cuda.set_device(3)

In [3]:
df_any.head()

,Modality,PatientID,StudyInstanceUID,SeriesInstanceUID,StudyID,ImagePositionPatient,ImageOrientationPatient,SamplesPerPixel,PhotometricInterpretation,Rows,...,MultiWindowCenter,WindowCenter1,MultiWindowWidth,WindowWidth1,any,epidural,intraparenchymal,intraventricular,subarachnoid,subdural
SOPInstanceUID,,,,,,,,,,,,,,,,,,,,,
ID_2a8702d25,CT,ID_ff137633,ID_d17053848c,ID_7098f7c836,,-126.437378,1.0,1,MONOCHROME2,512,...,NaN,NaN,NaN,NaN,1,0,1,1,0,0
ID_66891ac22,CT,ID_42940b2c,ID_17e33f43d0,ID_e14dd0090b,,-125.000000,1.0,1,MONOCHROME2,512,...,NaN,NaN,NaN,NaN,1,0,1,0,0,0
ID_8e6e5b51f,CT,ID_76fbed32,ID_1d8eaa14ef,ID_e3919709a0,,-125.000000,1.0,1,MONOCHROME2,512,...,NaN,NaN,NaN,NaN,1,0,0,0,0,1
ID_cb8b9b514,CT,ID_20039b63,ID_1cfe3e70dd,ID_e3b5d8d9b8,,-125.000000,1.0,1,MONOCHROME2,512,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0
ID_ee683911f,CT,ID_a065f3ac,ID_9c727ac231,ID_99e83a310d,,-125.000000,1.0,1,MONOCHROME2,512,...,1.0,40.0,1.0,80.0,0,0,0,0,0,0


In [4]:
fns = L(list(df_any.index))

In [5]:
def get_data(fns, bs, sz, img_dir='nocrop_jpg', splits=splits_any):
    if sz <= 256: img_dir = f'{img_dir}256'
    return get_data_gen(fns, bs=bs, img_tfm=get_pil_fn(path/img_dir), 
                        sz=sz, splits=splits)

## Training

In [6]:
name = 'baseline_any'

In [7]:
dbch = get_data(fns, 512, 128)
learn = get_learner(dbch, xresnet34)

## Training

In [8]:
do_fit(learn, 20, 4e-2, freeze=False)
learn.save(f'runs/{name}-1')

epoch,train_loss,valid_loss,accuracy_multi,accuracy_any,time
0,0.254301,0.278034,0.896199,0.806834,03:09
1,0.225084,0.367604,0.875990,0.712250,03:14
2,0.212654,0.495538,0.814444,0.497947,03:14
3,0.205387,0.299321,0.899139,0.812104,03:14
4,0.199100,0.270439,0.904211,0.796716,03:15
5,0.192158,0.209285,0.922880,0.861820,03:15
6,0.188286,0.195047,0.929555,0.876705,03:15
7,0.182425,0.199078,0.929281,0.868971,03:16
8,0.180373,0.205410,0.925202,0.864177,03:14
9,0.174530,0.194176,0.930446,0.878692,03:12


In [ ]:
learn.dbunch = get_data(fns, 512, 256)
do_fit(learn, 20, 2e-3, freeze=False)
learn.save(f'runs/{name}-2')

epoch,train_loss,valid_loss,accuracy_multi,accuracy_any,time
0,0.186483,0.195522,0.930711,0.877765,08:23
1,0.168234,0.178869,0.937002,0.888651,08:06
2,0.157464,0.170420,0.939505,0.894186,08:10
3,0.151394,0.167016,0.942048,0.897285,08:16
4,0.148513,0.165948,0.941443,0.897418,08:13
5,0.143086,0.167033,0.941509,0.896358,08:12
6,0.139229,0.165709,0.942445,0.896252,08:12
7,0.137324,0.166132,0.942767,0.898662,08:10
8,0.132522,0.169561,0.942824,0.898239,08:14
9,0.130888,0.168320,0.942048,0.897524,08:25


In [ ]:
learn.load(f'runs/{name}-2')

In [ ]:
fns_full = L(list(df_comb.index))
learn.dbunch = get_data(fns_full, 512, 256, splits=splits)
do_fit(learn, 2, 1e-4, freeze=False)
learn.save(f'runs/{name}-2b')

## Submission

In [19]:
learn.load(f'runs/{name}-3')
sub_fn = f'subm/{name}'

In [ ]:
learn.dbunch = get_test_data(df_tst, bs=1024, sz=224)
preds,targs = learn.get_preds()

In [33]:
pred_csv = submission(df_tst, preds, fn=nm)
FileLink(f'{sub_fn}.csv')

In [36]:
api.competition_submit(f'{nm}.csv', '0.0744,rn34_repl (scl 0.88)', 'rsna-intracranial-hemorrhage-detection')

100%|██████████| 26.0M/26.0M [00:02<00:00, 9.52MB/s]


Successfully submitted to RSNA Intracranial Hemorrhage Detection

In [ ]:
api.competitions_submissions_list('rsna-intracranial-hemorrhage-detection')[0]

## Export

In [3]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 01_data_01_metadata_stage2.ipynb.
Converted 01_data_02_preprocess_windows.ipynb.
Converted 02_train_01_train.ipynb.
Converted 04_orig_replace_ashaw_refactor.ipynb.
Converted 04_replace_ashaw_refactor.ipynb.
Converted 04b_orig_replace_ashaw_refactor.ipynb.
Converted 10_qure.ipynb.
Converted 12_merge.ipynb.
Converted 14_xgboost.ipynb.
Converted 16_slice_e2e-shallow.ipynb.
Converted 16b_orig_slice_e2e-shallow.ipynb.
This cell doesn't have an export destination and was ignored:
e
Converted 17_slice_model-deep.ipynb.
Converted 21_cleanup-nocrop2.ipynb.
Converted 26_submit_final.ipynb.
Converted 27_ensemble_tabular_nn.ipynb.
Converted 99_index.ipynb.
Converted cleanup-combine-qure.ipynb.
Converted delete_03b_cleanup-tif.ipynb.
Converted submit.ipynb.
Converted walkthru.ipynb.
Converted x00_tcia-ct-segm-prep.ipynb.
Converted x00_tcia-ct-segm-train.ipynb.
